# Quickstart

In the quickstart guide, we demonstrate how to create a basic data labeling agent.

Let's use a dataset of product reviews stored in pandas dataframe, and trying to understand subjectiveness of given texts:

In [24]:
import pandas as pd
texts = [
    "The mic is great.",
    "Will order from them again!",
    "Not loud enough and doesn't turn on like it should.",
    "The phone doesn't seem to accept anything except CBR mp3s",
    "All three broke within two months of use."
]
df = pd.DataFrame(texts, columns=['text'])
df

,text
0,The mic is great.
1,Will order from them again!
2,Not loud enough and doesn't turn on like it sh...
3,The phone doesn't seem to accept anything exce...
4,All three broke within two months of use.


Since the goal is to classify whether each review represents objective statement or subjective opionion, let's define our agent who should classify given dataset into the categories. The agent labeling abilities are defined as **"Skills"**, and each agent can possess many different skills. 

For simplicity, let's create an agent with only one labeling skill:

In [25]:
from adala.agents import SingleShotAgent
from adala.datasets import DataFrameDataset
from adala.skills import LabelingSkill

agent = SingleShotAgent(
    # connect to a dataset
    dataset=DataFrameDataset(df=df),
    
    # define the agent's labeling skill
    skill=LabelingSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective']
    )
)

agent.greet()


Hi, I am a single shot agent.
I can use a single skill to process a dataset.
For example, you can instruct me to label your dataset.
I have a skill name='subjectivity_detection' description='Understanding subjective and objective statements from text.' prompt_template="    {{>instructions}}\n    \n    Input: {{text}}\n    Output: {{select 'predictions' options=labels logprobs='score'}}\n    " instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.' prediction_field='predictions' labels=['Subjective', 'Objective'] which can label your dataset.


Now let's run our agent on a given dataset

In [26]:
run_results = agent.run()
pd.concat((df, run_results.experience.predictions), axis=1)

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 49.62it/s]


,text,ground_truth,predictions,score
0,The mic is great.,NaN,Subjective,"{'Subjective': -0.04570976299999997, 'Objectiv..."
1,Will order from them again!,NaN,Subjective,"{'Subjective': -0.31326162999999996, 'Objectiv..."
2,Not loud enough and doesn't turn on like it sh...,NaN,Subjective,"{'Subjective': -0.10020652400000002, 'Objectiv..."
3,The phone doesn't seem to accept anything exce...,NaN,Subjective,"{'Subjective': -0.674278, 'Objective': -0.7123..."
4,All three broke within two months of use.,NaN,Objective,"{'Subjective': -3.1679988, 'Objective': -0.042..."


We may observe some errors in the predicted dataset. Let's fix them and create _ground truth_ signal:

In [27]:
df.loc[0, 'ground_truth'] = 'Subjective'
df.loc[1, 'ground_truth'] = 'Subjective'
df.loc[2, 'ground_truth'] = 'Objective'
df.loc[3, 'ground_truth'] = 'Objective'
df.loc[4, 'ground_truth'] = 'Objective'

and then we can ask our agent to learn and improve itself:

In [28]:
learn_results = agent.learn()

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 54.83it/s]


Evaluating, analyzing and improving...
Done!


In [30]:
learn_results.experience.updated_instructions

'Classify a product review considering the following aspects: if the review consists of factual information or concrete feature descriptions, categorize it as "Objective"; if the review includes personal feelings, opinions, beliefs, or preferences of the reviewer, categorize it as "Subjective".\n\nExamples:\nInput: "The phone doesn\'t seem to accept anything except CBR mp3s."\nOutput: "Objective"\nInput: "Not loud enough and doesn\'t turn on like it should."\nOutput: "Objective"\nInput: "I don\'t like the color of this phone."\nOutput: "Subjective"'

In [1]:
import pandas as pd

from adala.agents import SingleShotAgent
from adala.datasets import DataFrameDataset
from adala.skills import LabelingSkill
from adala.runtimes import LLMRuntime, LLMRuntimeModelType

filepath = 'https://hs-sandbox-pub.s3.amazonaws.com/amazon_cells_labelled.tsv'
df = pd.read_csv(filepath, sep='\t', nrows=100)

agent = SingleShotAgent(
    
    # connect to a dataset
    dataset=DataFrameDataset(
        df=df,
        ground_truth_column='ground_truth'
    ),
    
    # define the labeling skill
    skill=LabelingSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective']
    ),
    
    # list all available runtimes
    runtimes={
        'openai': LLMRuntime(
            llm_runtime_type=LLMRuntimeModelType.OpenAI,
            llm_params={
                'model': 'gpt-3.5-turbo-instruct',
            }
        ),
        'openai-gpt4': LLMRuntime(
            llm_runtime_type=LLMRuntimeModelType.OpenAI,
            llm_params={
                'model': 'gpt-4',
            }
        ),
#         'llama2': LLMRuntime(
#             llm_runtime_type=LLMRuntimeModelType.Transformers,
#             llm_params={
#                 'model': 'meta-llama/Llama-2-7b',
#                 'device': 'cuda:0',
#             }
#         )
    },
    
    # use default runtime for the labeling skill
    default_runtime='openai'
)

Now we can run agent to retrieve predictions:

In [2]:
run_result = agent.run()

100%|█████████████████████████████████████████████| 20/20 [00:00<00:00, 76.97it/s]


In [3]:
pd.concat((df, run_result.experience.predictions), axis=1)

,text,label,ground_truth,predictions,score
0,So there is no way for me to plug it in here i...,0,NaN,Objective,"{'Subjective': -2.6367831, 'Objective': -0.074..."
1,"Good case, Excellent value.",1,NaN,Subjective,"{'Subjective': -0.6931471, 'Objective': -0.693..."
2,Great for the jawbone.,1,NaN,Objective,"{'Subjective': -0.9740768699999999, 'Objective..."
3,Tied to charger for conversations lasting more...,0,NaN,Subjective,"{'Subjective': -0.40735099999999996, 'Objectiv..."
4,The mic is great.,1,NaN,Subjective,"{'Subjective': -0.04570976299999997, 'Objectiv..."
...,...,...,...,...,...
95,Will order from them again!,1,NaN,Subjective,"{'Subjective': -0.31326162999999996, 'Objectiv..."
96,If you plan to use this in a car forget about it.,0,NaN,Objective,"{'Subjective': -0.75759894, 'Objective': -0.63..."
97,I found this product to be waaay too big.,0,NaN,Subjective,"{'Subjective': -0.01930922600000004, 'Objectiv..."
98,Best I've found so far .... I've tried 2 other...,1,NaN,Subjective,"{'Subjective': -0.07888966999999994, 'Objectiv..."


Now we can create a couple of ground truth samples to improve agent's skill:

In [4]:
df.loc[0, 'ground_truth'] = 'Objective'
df.loc[1, 'ground_truth'] = 'Subjective'
df.loc[3, 'ground_truth'] = 'Objective'
df.loc[4, 'ground_truth'] = 'Subjective'

Now ensure our agent learns from the inputs we've provided, by running a few iterations:

In [6]:
for iteration in range(3):
    print(f'Iteration #{iteration}')
    run_result = agent.run()
    learn_result = agent.learn(update_instructions=True)
    
    print(f'Accuracy: {learn_result.experience.accuracy}')

Iteration #0


100%|█████████████████████████████████████████████| 20/20 [00:00<00:00, 61.48it/s]


Evaluating, analyzing and improving...
Done!
Accuracy: 0.75
Iteration #1


100%|█████████████████████████████████████████████| 20/20 [00:00<00:00, 44.18it/s]


Evaluating, analyzing and improving...
Done!
Accuracy: 0.5
Iteration #2


100%|█████████████████████████████████████████████| 20/20 [00:00<00:00, 27.00it/s]


Evaluating, analyzing and improving...
Done!
Accuracy: 1.0


In [7]:
print(agent.skill.instructions)

Classify a product review as "Subjective" or "Objective". A review should be classified as "Subjective" if it consists of personal opinions, feelings, or experiences, whether positive or negative. Terms that could indicate subjectivity include evaluative adjectives expressing personal sentiment (e.g., "love", "like", "hate", "prefer"). Additionally, the phraseology suggesting a personal take, a recommendation, or a summary judgment by the reviewer would also be a key to subjectivity. Label a review as "Objective" if it merely presents straight facts about the product's features, specifications, or identified performance issues without expressing any personal affective sentiments or evaluative adjectives. Refrain from considering clear, factual statements of both positive or negative aspects of the product as subjective.

Examples:
Input: "The phone operates on Android 11."
Output: Objective
Input: "I really appreciate the long battery life of this phone."
Output: Subjective
Input: "The